
# Simulated Annealing of the Eggholder Function


## Importing Libraries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LinearLocator
from matplotlib import cm
from tqdm import tqdm
import matplotlib.gridspec as gridspec
from sys import path_importer_cache
import os
import eggholder as egg
import init_archive as inar


## Initial Temperature and Random Step Functions


In [ ]:
def init_temp_kirkpatrick(x0, fx0):
    """Estimates the initial temperature by conducting an initial search
    of M steps and accepting all of them. Then we take all steps that increased
    the objective function and calculated the mean increase df_pos. Then calculate 
    T0 = -(df_pos)/ln(chi_0) [Kirkpatrick, 1984].

    inputs: 
        x0: starting value of x
        fx0: function evaluated at x0

    returns estimated initial temperature T0
    """
    M = 1000
    chi_0 = 0.8
    
    f_array = []
    for i in range(M):
        x = random_step_basic(x0,d)
        fx = egg.eggholder(x)
        if fx > fx0:
            f_array.append(fx-fx0)
        x0 = x.copy()
        fx0 = fx.copy()
    df_pos = np.mean(f_array)
    T0 = -df_pos/np.log(chi_0)
    return T0

def init_temp_white(x0):
    """Estimates the initial temperature by conducting an initial search
    of M steps, accepting all random steps, and finding the standard deviation
    sigma of f across the steps. Then T0 = sigma [White, 1984]

    inputs: 
        x0: starting value x

    returns estimated initial temperature T0
    """
    M = 1000
    
    f_array = []
    for i in range(M):
        x = random_step_basic(x0,d)
        f_array.append(egg.eggholder(x))
        x0 = x.copy()
    T0 = np.std(f_array)
    return T0

def random_step_basic(x0, d, c=25, constrained = True):
    '''Taking a random step from point x0.
    
    inputs
        x0: point x we are taking the random step from
        d: dimension of x
        c: max change allowed of each variable.
        constrained: boolean for if using constraints or penalty functions.
    
    returns new point x.
    '''
    x = np.zeros(d)
    for i in range(d):
        new = x0[i] + np.random.uniform(-c, c)
        if constrained:
            while abs(new) > 512:
                new = x0[i] + np.random.uniform(-c, c)
        x[i] = new
    return x

def vanderbilt_recal(path, S0):
    '''Recalculating S and Q for taking random steps in the vanderbilt method.
    inputs
        path:   {fx:x}                  dict of all points accepted 
        S0:     (d,d)-length matrix     previous value of covariance matrix S
    
    returns recalculated Q and S.
    '''

    alpha = 0.1
    omega = 2.1
    if len(path) == 1:
        X = np.eye(d)
    else:
        path_values = np.array([*path.values()])
        X = np.cov(path_values.T, bias = True)

    S = (1-alpha)*S0 + alpha*omega*X
    try:
        Q = np.linalg.cholesky(S)
        return Q, S
    except np.linalg.LinAlgError:
        print("Cholesky decomposition failed.")
        raise ValueError


def random_step_vanderbilt(x0, d, Q, constrained = False):
    '''Taking a random step from point x0 using the vanderbilt method.
    
    inputs
        x0:             d-length vector         point x we are taking the random step from
        d:              int                     dimension of x
        Q:              (d,d)-length matrix     cholesky decomposition of the estimated covariance
        constrained:    boolean                 using constraints or penalty functions
    
    returns new point x, new covariance matrix S and a boolean on whether the 
    Cholesky decomposition has failed.
    '''

    u = (np.random.rand(d)*2-1)*np.sqrt(3)
    x_new = x0 + Q@u
    if constrained:
        count = 0
        count_limit = 5e3
        while not egg.feasible_check(x_new) and count < count_limit:
            u = np.random.rand(d)*np.sqrt(3)
            x_new = x0 + Q@u
            count += 1
        if count >= count_limit:
            raise RuntimeError
    return x_new


def parks_recal(u, D):
    '''Recalculating the D matrix for taking random steps using the Parks method.
    D is a diagonal matrix defining the max change for each dimension of x.
    inputs
        u:      d-length vector         previous random step taken 
        D:      (d,d)-length matrix     Controls max change in each dimension
    
    returns new calculated D and R.
    '''
    alpha = 0.1
    omega = 2.1

    R = np.abs(np.diag((D@u)))
    D_new = (1 - alpha) * D + alpha * omega * R

    return D_new, R


def random_step_parks(x0, d, D, constrained = False):
    '''Taking a random step from point x0 using the third method from lectures, proposed by 
    GT Parks in 1990.
    
    inputs
        x0:     d-length vector         point x we are taking the random step from
        d:      int                     dimension of x
        D:      (d,d)-length matrix     Matrix controlling max change in each dimension
    
    returns new point x and random step variable u.
    '''
    u  = np.random.rand(d)*2-1
    x_new = x0 + D.dot(u)

    if constrained:
        count_limit = 5e3
        count = 0
        while not egg.feasible_check(x_new) and count < count_limit:
            u  = np.random.rand(d)*2-1
            x_new = x0 + D.dot(u)
            count += 1
        if count >= count_limit:
            raise RuntimeError
            
    return x_new, u

def init_SA(d, seed, temp_method = 'Kirkpatrick'):
    '''Create starting parameters for SA run.'''
    x0 = inar.init_x(1, d)
    fx0 = egg.eggholder(x0)
    T0 = np.inf

    if temp_method == 'Kirkpatrick':
        T0 = init_temp_kirkpatrick(x0, fx0.copy())
    else:
        T0 = init_temp_white(x0)

    path = {fx0:x0.copy()}
    archive = {fx0:x0}  
    np.random.seed(seed)

    return x0, fx0, T0, path, archive


## Main Simulated Annealing Function (2D)

This is the initial implementation of the simulated annealing function for a 2D vector space, 
to check it is working properly and visualise the progression more easily.

In [ ]:
def SA_run(d = 2, step_method = 1, seed = 1, L_k = 500, c = 25, alpha = 0.99, 
        run_name = '1', temp_method = 'Kirkpatrick', constrained = True, w_val = 1e6):
    '''Conduct a run of Simulated Annealing under specified conditions.
    inputs
        d:              int     dimension of datapoints.
        step_method:    int     method used to take random steps
        seed:           float   random seed for different runs
        L_k:            int     length of Markov Chain
        c:              float   max step size for step method 1.
        alpha:          float   temperature change ratio
        run_name:       str     identifier for the run if required
        temp_method:    str     choose which method to use in initialising temperature
        constrained:    bool    decide whether to use active constraints or penalty functions

    returns the progress in best solution over iterations for the specified setting.
    '''
    x0, fx0, T0, path, archive = init_SA(d, seed, temp_method)

    current_best = fx0.copy() # this will be used to track restarts.
    x_best = x0.copy()
    new_best = fx0.copy() # this will be used to track restarts.
    x_old = x0.copy()
    fx_old = fx0.copy()

    # variables for different step methods.
    S = np.eye(d) * c
    D = np.eye(d) * c
    R = np.eye(d)

    f_evals_count = 0
    fx_progress = []
    T = T0
    restart_count = 0 # controls when to restart to the current best solution.
    while f_evals_count < 15000:
        if step_method == 2:
            try:
                Q, S = vanderbilt_recal(path, S)
            except ValueError:
                break
        n_min = 0.6*L_k # number of acceptances
        trial_count = 0 # controls number of iterations before adjusting the temperature
        acceptance_count = 0 # controls number of iterations before adjusting the temperature

        while trial_count < L_k and acceptance_count < n_min:
            fx_progress.append(current_best)
            trial_count += 1

            # generate a new solution
            if restart_count > 10000:
                restart_count = 0
                fx_old = current_best.copy()
                x_old = archive[fx_old].copy()
                break
            else:
                if step_method == 1:
                    x_new = random_step_basic(x_old,d, c, constrained)
                elif step_method == 2:
                    try:
                        x_new = random_step_vanderbilt(x_old, d, Q, constrained)
                    except RuntimeError:
                        print('failed to find feasible point.')
                        break
                else:
                    try:
                        x_new, u = random_step_parks(x_old, d, D, constrained)
                    except RuntimeError:
                        print('failed to find feasible point.')
                        break
                if constrained:
                    fx_new = egg.eggholder(x_new)
                else:
                    fx_new = egg.eggholder_unconstrained(x_new, w_val, T)
                f_evals_count += 1

            # assess the solution and choose whether to accept.
            accept = False
            del_f = fx_new - fx_old
            if del_f < 0:
                accept = True
            elif step_method == 3:
                d_bar = 0
                for i in range(len(x0)):
                    d_bar += R[i,i]**2
                d_bar = np.sqrt(d_bar)
                p = np.exp(-del_f/(T*d_bar))
                accept = np.random.rand() < p
            else:
                p = np.exp(-del_f/T)
                accept = np.random.rand() < p

            # if accepting, try archiving.
            if accept:
                acceptance_count += 1
                archive, updated = inar.update_archive(x_new, fx_new, archive)
                if updated:
                    new_best = min([*archive]).copy()
                path[fx_new] = x_new.copy()
                x_old = x_new.copy()

                if step_method == 3:
                    D, R = parks_recal(u, D)

            if new_best == current_best:
                restart_count += 1
            elif new_best < current_best:
                current_best = new_best.copy()
                x_best = archive[current_best].copy()
            else:
                print('failed')
                raise ValueError
        # adjust the temperature
        T *= alpha
        if acceptance_count == 0:
            break

    fx_star = min([*archive])
    print("\nBest solution: \n", fx_star, archive[fx_star])  
    print(len(path))
    print(len(archive))
    if step_method == 2:
        print('Q: ',Q)
    elif step_method == 3:
        print('D: ',D)
    z = []
    if d==2:
        # Draw contour plot and evolution of best solution found across iterations
        z = egg.plot_eggholder2D(archive, path, run_name)
    egg.plot_progress(fx_progress, z, d, run_name)
    # return fx_progress


In [ ]:
d = 6
SA_run(d,1,1,500,50,0.9,'unconstrained','Kirkpatrick',False, 1e6)


## Large Runs



### Varying Just the Seed

In [ ]:
d=6

fx_progress_list = []
for j in tqdm(range(100)):
    fx_progress_list.append(SA_run(d, 1, j, 500, 25, 0.9, 'unconstrained', 'Kirkpatrick', False, 1e9))

egg.plot_progress_multseed(fx_progress_list, d, title = '1 billion')



### Temperature Initialisation Methods

In [ ]:

fx_progress_dict = {}
alpha = 0.9
for method in ['White', 'Kirkpatrick']:
    for d in [2,6]:
        graph_name = '{}_{}D'.format(method, d)
        print(graph_name)
        progress_list = []
        for i in range(60):
            progress_list.append(SA_run(d, 1, i, 500, 25, alpha, 'temp_comparisons', method, True))
        fx_progress_dict[graph_name] = progress_list
    
egg.plot_progress_multvar(fx_progress_dict, d, filename='Temp_init_comparisons_alpha={}'.format(alpha))



### Alpha Variation

In [ ]:

fx_progress_dict = {}
for alpha in [0.85, 0.9, 0.95, 0.99]:
    graph_name = alpha
    print(graph_name)
    progress_list = []
    for i in tqdm(range(100)):
        progress_list.append(SA_run(d, 1, i, 500, 25, alpha, 'temp_comparisons', 'Kirkpatrick', True))
        fx_progress_dict[graph_name] = progress_list
    
egg.plot_progress_multvar(fx_progress_dict, d, filename='Alpha_comparisons_Kirkpatrick')



### Investigating Unconstrained

In [ ]:
d = 6
fx_progress_dict = {}
for w_val in [0, 1e3, 1e5, 1e6, 1e9]:
    graph_name = w_val
    if w_val == 0:
        constrained = True
        graph_name = 'Constrained'
    else:
        constrained = False
    print(graph_name)
    progress_list = []
    for i in tqdm(range(100)):
        progress_list.append(SA_run(d, 1, i, 500, 25, 0.9, 'unconstrained_comparisons', 'Kirkpatrick', constrained, w_val))
        fx_progress_dict[graph_name] = progress_list
    
egg.plot_progress_multvar(fx_progress_dict, d, filename='Unconstrained_Comparison_2')


### Investigating Step Size

In [ ]:
d = 6
fx_progress_dict = {}
for step_size in [25, 50, 75, 100]:
    graph_name = 'max_step: {}'.format(step_size)
    print(graph_name)
    progress_list = []
    for i in tqdm(range(100)):
        progress_list.append(SA_run(d, 1, i, 500, step_size, 0.9, 'unconstrained_comparisons', 'Kirkpatrick', False, 1e6))
        fx_progress_dict[graph_name] = progress_list
    
egg.plot_progress_multvar(fx_progress_dict, d, filename='Step_Size_Comparison')


### Investigating Markov Chain Length

In [ ]:
d = 6
fx_progress_dict = {}
for chain_length in [1000, 1500, 2000, 2500]:
    graph_name = chain_length
    print(graph_name)
    progress_list = []
    for i in tqdm(range(100)):
        progress_list.append(SA_run(d, 1, i, chain_length, 50, 0.8, 'unconstrained_comparisons', 'Kirkpatrick', False, 1e5))
        fx_progress_dict[graph_name] = progress_list
    
egg.plot_progress_multvar(fx_progress_dict, d, filename='Markov_Chain_Length_Comparison_2')


### Investigating Random Step Methods

In [ ]:
d = 6
constrained = False
fx_progress_dict = {}
for index,step_method in enumerate(['Vanderbilt', 'Parks']):
    for w_val in [1e5, np.inf]:
        title_edit = ''
        if w_val == np.inf:
            title_edit = '*'
            if step_method == 'Parks':
                constrained = True
        graph_name = '{}{}'.format(step_method, title_edit)
        print(graph_name)
        progress_list = []
        for i in tqdm(range(100)):
            progress_list.append(SA_run(d, (index+2), i, 500, 100, 0.9, 'unconstrained_comparisons', 'Kirkpatrick', constrained, w_val))
            fx_progress_dict[graph_name] = progress_list
    
egg.plot_progress_multvar(fx_progress_dict, d, filename='Step_Methods_Comparison_2')


### Investigating Parks Method

In [ ]:
d = 6
fx_progress_dict = {}
for step_size in [25, 50, 100, 200]:
    graph_name = 'init_step: {}'.format(step_size)
    print(graph_name)
    progress_list = []
    for i in tqdm(range(100)):
        progress_list.append(SA_run(d, 3, i, 500, step_size, 0.95, 'unconstrained_comparisons', 'White', True))
        fx_progress_dict[graph_name] = progress_list
    
egg.plot_progress_multvar(fx_progress_dict, d, filename='Parks_Step_size_constrained_white')


### MC Length with Alpha

In [ ]:
d = 6
fx_progress_dict = {}
for chain_length in [100, 1500]:
    for alpha in [0.8, 0.99]:
        graph_name = 'MC: {}, a: {}'.format(chain_length, alpha)
        print(graph_name)
        progress_list = []
        for i in tqdm(range(100)):
            progress_list.append(SA_run(d, 1, i, chain_length, 50, alpha, 'unconstrained_comparisons', 'Kirkpatrick', False, 1e5))
            fx_progress_dict[graph_name] = progress_list
    
egg.plot_progress_multvar(fx_progress_dict, d, filename='MC_alpha_cross_2')